# OIS Yield Curve Construction 

Constructing the OIS yield curve can be done via mid-quotes of market makers. The OIS curve can be constructed entirely from OIS instruments. For this yield curve construction we aim to find the OIS forward rate which can be found by discritizing the forward curve into daily rates, and fitting these to the zero-rates found by bootstrapping.

Thus, the construction is based on optimzation and not interpolation. 

In [2]:
import numpy as np
import pandas as pd

### Constructing Yield Curve without Regularization

### Constructing Yield Curve with Regularization

### PCA: Yield Curve Driving Factors